In [2]:
!nvidia-smi

Tue Apr  6 04:55:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# INSTALL

In [4]:
!ln -s '/content/drive/MyDrive/THESIS/AI_CITY_2021/SOURCE' '/content'
!ln -s '/content/drive/MyDrive/THESIS/AI_CITY_2021/DATA' '/content'

In [ ]:
# %%capture 
!git clone https://github.com/lukemelas/EfficientNet-PyTorch.git
%cd /content/EfficientNet-PyTorch
!pip install -e .

# DEPENDENCIES

In [6]:
import os, json, cv2
import os.path as osp 
import pandas as pd 
from tqdm.notebook import tqdm
from glob import glob
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder

import pandas as pd 
import cv2, json, os, shutil
import numpy as np 
import matplotlib.pyplot as plt
import os.path as osp
from tqdm.notebook import tqdm
from glob import glob
from google.colab.patches import cv2_imshow

import torch
import torch.nn as nn 
from torch.utils.data import Subset, DataLoader, Dataset

from efficientnet_pytorch import EfficientNet

import PIL
import pickle

# CONFIGURATION

In [7]:
TRACKS_DIR = '/content/SOURCE/vehicle_tracking/data/Centernet2_train_veh_boxes.json'
ROOT_DIR = '/content/DATA/data_track_5/AIC21_Track5_NL_Retrieval'

In [ ]:
CONFIG_VEHICLE = {
    'MODEL': 'efficientnet-b5',
    'SAVE_DIR': '/content/SOURCE/extract_object/results/vehicle_classification_fraction',
    'SAVE_FEATURE_DIR': '/content/SOURCE/nanonets_object_tracking/feature/vehicle',
    'EXP_ID': 'Mar25_b5_datav2',
    'NUM_CLASSES': 6,
    'image_size': (224,224),
    "imagenet_mean":[0.485, 0.456, 0.406],
    "imagenet_std":[0.229, 0.224, 0.225],
    'score_thres': 0.5,
    'seed': 88,
}

CONFIG_VEHICLE['SAVE_DIR'] = osp.join(CONFIG_VEHICLE['SAVE_DIR'], CONFIG_VEHICLE['EXP_ID'])
os.makedirs(CONFIG_VEHICLE['SAVE_DIR'], exist_ok=True)

# MODEL

In [ ]:
class Classifier(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        backbone = EfficientNet.from_pretrained(cfg['MODEL'], include_top=False)
        # self.backbone = self.backbone.cuda()
        out_channel = backbone._conv_head.out_channels

        self.feature_extractor = nn.Sequential(
            backbone,
            nn.Flatten()
        )
        # avg_pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(out_channel, cfg['NUM_CLASSES'])
        )
        # self.classifier = nn.Linear(out_channel, cfg['NUM_CLASSES'])
        
    
    def extract_feature(self, input):
        x = self.feature_extractor(input)
        # x = self.avg_pool(x)
        return x
        
    def forward(self, input):
        x = self.extract_feature(input)
        logits = self.classifier(x)
        # logits = self.skeleton(input)
        logits = torch.softmax(logits, dim=-1)
        return logits
        

## Sanity check

In [ ]:
model = Classifier(CONFIG_VEHICLE)
model = model.cuda()
a = torch.rand((1, 3, 224, 224)).cuda()
a_feats = model.extract_feature(a)
a_pred = model(a)
print(f'feat shape: {a_feats.shape}')
print(f'pred shape: {a_pred.shape}')
print(a_feats.dtype)
print(a_pred.dtype)
print(a_feats.get_device())
print(a_pred.get_device())

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth



Loaded pretrained weights for efficientnet-b5
feat shape: torch.Size([1, 2048])
pred shape: torch.Size([1, 6])
torch.float32
torch.float32
0
0


# DATASET

In [ ]:
track = json.load(open(TRACKS_DIR))

# CHECKPOINT

In [ ]:
model_vehicle = Classifier(CONFIG_VEHICLE)

ckpt_path = osp.join(CONFIG_VEHICLE['SAVE_DIR'], 'best_model.pt')
if osp.isfile(ckpt_path):
    model_vehicle.load_state_dict(torch.load(ckpt_path))
    print(f'Loaded checkpoint from {ckpt_path}')

model_vehicle = model_vehicle.cuda()
model_vehicle.eval()

# INFERENCE

In [ ]:
val_transform = transforms.Compose([
    transforms.Resize(CONFIG_VEHICLE['image_size'], PIL.Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(CONFIG_VEHICLE['imagenet_mean'], CONFIG_VEHICLE['imagenet_std']),
])

def preprocess_input(img):
    img = img.convert('RGB')
    img = val_transform(img)
    return img

In [ ]:
for track_id in tqdm(track):
    ans = {}
    ans[track_id] = {}
    for val in tqdm(track[track_id]):
        for key in val:
            ans[track_id][key] = []
            new_path = os.path.join(ROOT_DIR, key)
            img = Image.open(new_path)
            for box in val[key]:
                x_0, y_0, x_1, y_1 = box
                x_0, y_0, x_1, y_1 = int(x_0), int(y_0), int(x_1), int(y_1)
                crop_img = img.crop((x_0, y_0, x_1, y_1))
                crop_img = preprocess_input(crop_img).cuda()
                crop_img = crop_img.unsqueeze(0)
                with torch.no_grad():
                    feature = model_vehicle.extract_feature(crop_img)
                ans[track_id][key].append(feature)
    filename = f'{track_id}.pickle'
    save_dir = os.path.join(CONFIG_VEHICLE['SAVE_FEATURE_DIR'], filename)
    with open(save_dir, 'wb') as handle:
        pickle.dump(ans, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Linh tinh

In [11]:
!pip install tqdm -q

In [12]:
import os
from tqdm.notebook import tqdm
root = "/content/DATA/data_track_5/AIC21_Track5_NL_Retrieval/validation/S05"
list_cam = os.listdir(root)

In [13]:
value = {}

for cam in tqdm(list_cam):
    path = os.path.join(root, cam, "img1")
    ans = len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])
    value[cam] = ans

In [33]:
value

{'c010': 4072,
 'c016': 3940,
 'c017': 3879,
 'c018': 3844,
 'c019': 3897,
 'c020': 3973,
 'c021': 4001,
 'c022': 4277,
 'c023': 4255,
 'c024': 4299,
 'c025': 4280,
 'c026': 4177,
 'c027': 3845,
 'c028': 3825,
 'c029': 3545,
 'c033': 3407,
 'c034': 3425,
 'c035': 3472,
 'c036': 3432}

In [27]:
num = 15
extend = '0'*(6-len(str(num)))
name = f'{extend}{num}.jpg'
name

'000015.jpg'

In [18]:
import shutil

def ig_f(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir, f))]

SRC = "/content/DATA/data_track_5/AIC21_Track5_NL_Retrieval/validation/S05"
DES = "/content/SOURCE/temp_missing_files"
shutil.copytree(SRC, DES, ignore=ig_f)

'/content/SOURCE/temp_missing_files'

In [19]:
SRC = "/content/DATA/data_track_5/AIC21_Track5_NL_Retrieval/validation/S05"
DES = "/content/SOURCE/temp_missing_files/S05"

In [20]:
import json
missing = json.load(open("/content/SOURCE/temp_missing_files/missing.json"))

In [32]:
from shutil import copyfile

for cam in tqdm(list_cam):
    src_path = os.path.join(SRC, cam)
    des_path = os.path.join(DES, cam)
    for val in tqdm(missing[cam]):
        extend = '0'*(6-len(str(val)))
        name = f'{extend}{val}.jpg'
        copyfile(os.path.join(src_path,"img1",name), os.path.join(des_path,"img1",name))